In [ ]:
#export
import csv
import numpy as np  # http://www.numpy.org
import ast
from datetime import datetime
from math import log, floor, ceil
import random
import numpy as np

Modify the Utility class's methods. You can also add additional methods as required but don't change existing methods' arguments.

In [ ]:
#export
class Utility(object):
    
    # This method computes entropy for information gain
    def entropy(self, class_y):
        entropy = 0
        zeros = len([x for x in class_y if x == 0])
        ones  = len([x for x in class_y if x == 1])
        tots = len(class_y)
        
        if zeros > 0 and ones > 0:
          entropy = (-1) *(zeros / tots)* (log(zeros / tots, 2)) + (-1) * (ones / tots) * (log(ones / tots, 2)) 

        if zeros > 0 and ones == 0:
          entropy = (-1) *(zeros / tots)* (log(zeros / tots, 2))

        if zeros == 0 and ones > 0:
          entropy = (-1) * (ones / tots) * (log(ones / tots, 2)) 
        pass
        return entropy


    def partition_classes(self, X, y, split_attribute, split_val):

        X_left = []
        X_right = []

        y_left = []
        y_right = []

        combos = [[x,y] for x,y in zip(X,y)]
        combos_left = [x for x in combos if x[0][split_attribute] <= split_val]
        combos_right = [x for x in combos if x[0][split_attribute] > split_val]

        X_left = [x[0] for x in combos_left]
        X_right = [x[0] for x in combos_right]

        y_left = [x[1] for x in combos_left]
        y_right = [x[1] for x in combos_right]

        pass
      
        return (X_left, X_right, y_left, y_right)


    def information_gain(self, previous_y, current_y):
        info_gain = 0
        current_y_left = current_y[0]
        current_y_rigth = current_y[1]
        leftlen  = len(current_y_left)
        rigthlen = len(current_y_rigth)
        totlen = leftlen + rigthlen 

        info_gain = self.entropy(previous_y) - (self.entropy(current_y_left) * leftlen / totlen + self.entropy(current_y_rigth) * rigthlen / totlen)

        pass
        return info_gain


    def best_split(self, X, y):
        split_attribute = 0
        split_val = 0
        X_left, X_right, y_left, y_right = [], [], [], []

        colNum = len(X[0])                          # number of columns in the X
        split_vals_all = []                         # collect each column`s unique split values
        for i in range(0, colNum):
          split_vals   = [x[i] for x in X]
          split_vals_u = []
          [split_vals_u.append(x) for x in split_vals if x not in split_vals_u]
          split_vals_all.append(split_vals_u)       # collect each column`s uique split values

        info_gains_all = []
        for i in range(0, colNum):
          split_vals = split_vals_all[i]
          for split_val in split_vals:
            (X_left, X_right, y_left, y_right) = self.partition_classes(X, y, i, split_val)
            info_gains_all.append([X_left, X_right, y_left, y_right, self.information_gain(y, [y_left, y_right]), i, split_val])

        info_gains_all.sort(key=lambda x: x[4], reverse=True) 
      
        returnDict = {  "split_attribute": info_gains_all[0][5],
                        "split_val": info_gains_all[0][6],
                        "X_left": info_gains_all[0][0],
                        "X_right": info_gains_all[0][1],
                        "y_left": info_gains_all[0][2],
                        "y_right": info_gains_all[0][3],
                        "info_gain": info_gains_all[0][4] }
        pass
        return returnDict
        #############################################

### Define the classes 'DecisionTree' and 'RandomForest'

In [ ]:
#export
class DecisionTree(object):
    def __init__(self, max_depth):
        # Initializing the tree as an empty dictionary or list, as preferred
        self.tree = {}
        self.max_depth = max_depth
        
    	
    def learn(self, X, y, par_node = {}, depth=0):
     
        y_u = []
        [y_u.append(x) for x in y if x not in y_u]
        
        if len(y_u) == 0:
          self.tree['diagnosis'] = 0
          return 
        elif len(y_u) == 1:
          self.tree['diagnosis'] = y[0]
          return


        depth = depth + 1
        if self.max_depth < depth:
          return
        
        UtilityCall = Utility()
        bestSplit = UtilityCall.best_split(X, y)               # splits to X_left, y_left, X_right, y_right.

        self.tree['dt_left'] = DecisionTree(max_depth = self.max_depth - depth)
        self.tree['dt_left'].learn(bestSplit['X_left'], bestSplit['y_left'])

        self.tree['dt_right'] = DecisionTree(max_depth = self.max_depth - depth)
        self.tree['dt_right'].learn(bestSplit['X_right'], bestSplit['y_right'])
        
        self.tree['attribute'] = bestSplit['split_attribute']
        self.tree['value'] = bestSplit['split_val'] 
        pass



    def classify(self, record):
        treenode = self.tree
       
        while len(treenode) > 1:
            treenodeAtt = treenode['attribute']
            treenodeVal = treenode['value']

            if treenodeVal >= record[treenodeAtt]:
              treenode = treenode['dt_left'].tree
            else:
              treenode = treenode['dt_right'].tree

        return treenode['diagnosis']         
        pass


In [ ]:
class RandomForest(object):
    num_trees = 0
    decision_trees = []
    bootstraps_datasets = []

    bootstraps_labels = []

    def __init__(self, num_trees):
        # Initialization done here
        self.num_trees = num_trees
        self.decision_trees = [DecisionTree(max_depth=10) for i in range(num_trees)]
        self.bootstraps_datasets = []
        self.bootstraps_labels = []
        
    def _bootstrapping(self, XX, n):
        sample = []                                                                # sampled dataset
        labels = []                                                                # class labels for the sampled records
        XXRowNums = len(XX)                                                        # number of rows in the XX. 
        randRowNums = np.random.choice(list(range(0, XXRowNums)),n)                # randomly generated row indexes  
        
        sample = [XX[x][:-1] for x in randRowNums]                                 # sampled dataset
        labels = [XX[x][-1] for x in randRowNums]                                  # class labels for the sampled records            
        pass
        return (sample, labels)

    def bootstrapping(self, XX):
        # Initializing the bootstap datasets for each tree
        for i in range(self.num_trees):
            data_sample, data_label = self._bootstrapping(XX, len(XX))
            self.bootstraps_datasets.append(data_sample)
            self.bootstraps_labels.append(data_label)

    def fitting(self):
        [i.learn(self.bootstraps_datasets[j], self.bootstraps_labels[j]) for i,j in zip(self.decision_trees, list(range(0, len(self.decision_trees))))]       
        pass


    def voting(self, X):
        y = []

        for record in X:
            votes = []
            
            for i in range(len(self.bootstraps_datasets)):
                dataset = self.bootstraps_datasets[i]
                
                if record not in dataset:
                    OOB_tree = self.decision_trees[i]
                    effective_vote = OOB_tree.classify(record)
                    votes.append(effective_vote)

            counts = np.bincount(votes)

            if len(counts) == 0:
                votes = []
                [votes.append(x.classify(record)) for x in self.decision_trees]

                onesCount = len([x for x in votes if x == 1])      # number of ones
                zerosCount = len([x for x in votes if x == 0])     # number of zeros
                if onesCount >= zerosCount:
                  y = np.append(y, onesCount)
                else:
                  y = np.append(y, zerosCount)                
                pass
            else:
                y = np.append(y, np.argmax(counts))
                
        return y

    def user(self):
        return 'UnalUnsal'

In [ ]:

def get_forest_size():
    forest_size = 10
    return forest_size
def get_random_seed():
    random_seed = 0
    return random_seed

### Do not modify the below cell
The cell below is provided to test that your random forest classifier can be successfully built and run.

In [ ]:
def run():
    np.random.seed(get_random_seed())
    # start time 
    start = datetime.now()
    X = list()
    y = list()
    XX = list()  # Contains data features and data labels
    numerical_cols = set([i for i in range(0, 31)])  # indices of numeric attributes (columns)

    # Loading data set
    print("reading the data")
    with open("Wisconsin_breast_prognostic.csv") as f:
        next(f, None)
        for line in csv.reader(f, delimiter=","):
            xline = []
            for i in range(len(line)):
                if i in numerical_cols:
                    xline.append(ast.literal_eval(line[i]))
                else:
                    xline.append(line[i])

            X.append(xline[:-1])
            y.append(xline[-1])
            XX.append(xline[:])

    # Initializing a random forest.
    randomForest = RandomForest(get_forest_size())

    # printing the name
    print("__Name: " + randomForest.user()+"__")

    # Creating the bootstrapping datasets
    print("creating the bootstrap datasets")
    randomForest.bootstrapping(XX)

    # Building trees in the forest
    print("fitting the forest")
    randomForest.fitting()

    # Calculating an unbiased error estimation of the random forest
    # based on out-of-bag (OOB) error estimate.
    y_predicted = randomForest.voting(X)

    # Comparing predicted and true labels
    results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

    # Accuracy
    accuracy = float(results.count(True)) / float(len(results))

    print("accuracy: %.4f" % accuracy)
    print("OOB estimate: %.4f" % (1 - accuracy))

    # end time
    print("Execution time: " + str(datetime.now() - start))

In [ ]:
# Call the run() function to test your implementation
# Use this cell and any cells below for additional testing
run()